In [9]:
from numpy import genfromtxt
import numpy as np
import matplotlib.pyplot as plt

def readDataAndLabels(fileName):
    train = genfromtxt(fileName, delimiter=',', dtype=float, skip_header=1)
    return (train[:, 0], train[:, 1:])

def oneHot(array):
    m = np.max(array)
    ar = np.zeros((len(array), m+1))
    y=0
    for x in array:
        ar[y, x] = 1
        y+=1
    return ar


class NeuralNet:
    def __init__(self, layerSizes, activationFunction):
        self._layerSizes = layerSizes
        self._activationFunction = activationFunction
        self.initRandomWeights()
        
    def initRandomWeights(self):
        self._weights = []
        for layer in range(len(self._layerSizes)-1):
            self._weights.append(np.random.randn(self._layerSizes[layer], self._layerSizes[layer+1]))
        
        self._biases = []
        for bias in range(len(self._layerSizes)-1):
            self._biases.append(np.random.randn(1, self._layerSizes[bias+1]))
            
    def getWeights(self):
        return (self._weights, self._biases)
    
    #def getWeightsWithBiases(self):
    #    weights = list()
    #    for w,b in zip(self._weights, self._biases):
    #        weights.append(np.concatenate((b, w), axis=0))
    #    return weights
    
    def setWeights(self, weights, biases):
        self._weights = weights
        self._biases = biases
    
    #def forward(self, X):
    #    zlist, aList = [],[X]
    #    for weight, biases in zip(self._weights, self._biases):
    #        X = X.dot(weight) + biases
    #        zlist.append(X)
    #        X = self._activationFunction.value(X)
    #        aList.append(X)
    #    return (X, zlist, aList)
    
    def forward(self, X):
        for weight, biases in zip(self._weights, self._biases):
            X = X.dot(weight) + biases
            X = self._activationFunction.value(X)
        return X
    
    def predict(self, X):
        for weight, biases in zip(self._weights, self._biases):
            X = self._activationFunction.value(X.dot(weight) + biases)
        return X 
    
    def addOnesForBiasCalculation(self, X):
        z = np.ones((len(X),1))
        return np.concatenate((z, X), axis=1)
    
    def forwardTest(self, X):
        self.z2 = np.dot(X, self._weights[0])
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self._weights[0])
        yHat = self.sigmoid(self.z3) 
        return yHat
    
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)

    
    def backpropagation(self, X, Y, costFunction, learningRate = 0.0001, callBackFunction = None):
        
        
        self.yHat = self.forwardTest(X)
        
        callBackFunction(self.yHat, Y)
        
        delta3 = np.multiply(-(Y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        self._weights[0] -= dJdW1
        self._weights[1] -= dJdW2
        
        #return dJdW1, dJdW2
        
        
        #pred, zList, aList = self.forward(X)
        #
        #if callBackFunction is not None:
        #    predY, _, _ = self.forward(X)
        #    callBackFunction(predY, Y)
        #
        #error = costFunction.cost(pred, Y)
        #print(error)
        #zList = zList[::-1]
        #aList = aList[::-1]

        #delta3 = costFunction.delta(pred, Y, zList[0], self._activationFunction)
        #dJdW2 = np.dot(aList[1].T, delta3)
        #
        #delta2 = np.dot(delta3, self._weights[1].T)*self._activationFunction.derivative(zList[1])
        #dJdW1 = np.dot(X.T, delta2)
        #
        ##print(X.T.shape, aList[1].T.shape)
        #
        #self._weights[1] -= learningRate * dJdW2
        #self._weights[0] -= learningRate * dJdW1
        #
        #self._biases[1] -= learningRate * np.sum(delta3, axis=0, keepdims=True)
        #self._biases[0] -= learningRate * np.sum(delta2, axis=0, keepdims=True)

In [2]:
class SigmoidActivationFunction:
    def value(self, x):
        return 1/(1 + np.exp(-x))

    def derivative(self, x):
        v = self.value(x)
        return np.multiply(v, 1-v)

Sigmoidna funkcija in njen odvod

In [3]:
#sigX =  np.arange(-10, 10, 0.1)
#sigF = SigmoidActivationFunction()
#plt.plot(sigX, sigF.value(sigX), linewidth=1.4)
#plt.plot(sigX, sigF.derivative(sigX), linewidth=1.4)
#plt.show()


\begin{align}
J
&= \left[ \frac{1}{m} \sum_{i=1}^m \left( \frac{1}{2} \left\| h_{W,b}(x^{(i)}) - y^{(i)} \right\|^2 \right) \right]
                       + \frac{\lambda}{2} \sum_{l=1}^{n_l-1} \; \sum_{i=1}^{s_l} \; \sum_{j=1}^{s_{l+1}} \left( W^{(l)}_{ji} \right)^2
\end{align}


In [4]:
class MSECost:
    def cost(self, predicted, actual):
        return 0.5 * np.sum(np.power(predicted - actual, 2))/len(predicted)
    
    def delta(self, predicted, actual, z, activationFunction):
        return np.multiply(-(actual-predicted), activationFunction.derivative(z))
    
class PredictionAccuracy:
    @staticmethod
    def CA(predicted, actual): #OneHot encoded data
        neki = 12

In [5]:
datalabels, datapixels = readDataAndLabels('Data/train.csv')

In [10]:
def callBackFunction(predicted, actual):
    n = np.argmax(predicted, axis=1) - np.argmax(actual, axis=1)
    #print(predicted[1,:], actual[1,:], n[1])
    print("CA", (len(n) - np.count_nonzero(n))/len(n))

neuralNetwork = NeuralNet([784, 100, 10], SigmoidActivationFunction())
#pr, zList, aList = neuralNetwork.forward(datapixels/255)
actual = oneHot(datalabels.astype(int))
    

for x in range(50):
    neuralNetwork.backpropagation(datapixels/255, actual, MSECost(), 0.1, callBackFunction)

ValueError: shapes (42000,100) and (784,100) not aligned: 100 (dim 1) != 784 (dim 0)